In [ ]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
print("creating necessary folders...")
# Créer les dossiers nécessaires pour sauvegarder les résultats
base_dir = "embedding_results_stats_progressive"
os.makedirs(base_dir, exist_ok=True)
embedding_dir = os.path.join(base_dir, "embeddings")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(embedding_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
print("loading distances matrix...")
# Charger la matrice de distances
distance_matrix_path = 'jaccardIndex/jaccard_overall.csv'
distance_matrix = pd.read_csv(distance_matrix_path, low_memory=False)
distance_matrix.set_index('Unnamed: 0', inplace=True)
distance_matrix = distance_matrix.astype(float)
print("verifications...")
# Vérification si la matrice est carrée et symétrique
assert distance_matrix.shape[0] == distance_matrix.shape[1], "La matrice doit être carrée."
assert np.allclose(distance_matrix, distance_matrix.T), "La matrice doit être symétrique."
print("defining sizes...")
# Définir les tailles progressives (10, 100, 1000, etc.)
sizes = [10, 100, 1000, 10000]
if distance_matrix.shape[0] > 10000:
    sizes += list(range(20000, distance_matrix.shape[0] + 1, 20000))
else:
    sizes.append(distance_matrix.shape[0])  # Ajouter la taille totale si elle est inférieure à 1000

# Liste pour sauvegarder les temps d'exécution
execution_times = []

# Fonction pour mesurer et sauvegarder les résultats
def compute_mds_for_subset(size):
    print("computing for",size,"publications")
    print(f"Processing with {size} journals...")
    subset_matrix = distance_matrix.iloc[:size, :size]  # Sous-matrice carrée
    start_time = time.time()

    # Calculer l'embedding MDS
    mds = MDS(n_components=2, dissimilarity='precomputed', metric=False, random_state=42)
    embedding = mds.fit_transform(subset_matrix.values)
    elapsed_time = time.time() - start_time

    # Sauvegarder les résultats
    embedding_df = pd.DataFrame(embedding, index=subset_matrix.index, columns=["Dim_1", "Dim_2"])
    embedding_path = os.path.join(embedding_dir, f"embedding_MDS_{size}journals.csv")
    embedding_df.to_csv(embedding_path)
    
    print(f"Finished {size} journals in {elapsed_time:.2f} seconds.")
    return size, elapsed_time
print("starting...")
# Calculer MDS pour chaque taille définie
for size in sizes:
    size, elapsed_time = compute_mds_for_subset(size)
    print(elapsed_time)
    execution_times.append((size, elapsed_time))

# Convertir les résultats en DataFrame
times_df = pd.DataFrame(execution_times, columns=["Number of Journals", "Execution Time (s)"])

# Sauvegarder les temps d'exécution
times_path = os.path.join(base_dir, "execution_times.csv")
times_df.to_csv(times_path, index=False)

# Générer un graphique des temps d'exécution
plt.figure(figsize=(10, 6))
plt.plot(times_df["Number of Journals"], times_df["Execution Time (s)"], marker='o', linestyle='-')
plt.title("Temps d'exécution en fonction du nombre de journaux")
plt.xlabel("Nombre de journaux")
plt.ylabel("Temps d'exécution (secondes)")
plt.grid(True)

# Sauvegarder et afficher le graphique
graph_path = os.path.join(plots_dir, "execution_time_graph.png")
plt.savefig(graph_path)
plt.show()

print(f"Résultats sauvegardés dans {base_dir}.")

creating necessary folders...
loading distances matrix...
